本节以天绘二号影像为例，对完整的流程进行讲解。首先可以取一景TerraSAR进行实验，得到头文件，之后修改头文件得到天绘二号影像的头文件。

In [ ]:
from isce.components.isceobj.Util.Simamplitude import Simamplitude
from isce.components.isceobj import createImage
from isce.components.iscesys.StdOEL.StdOELPy import create_writer
from isce.components.isceobj import createOffoutliers
from isce.components.mroipac.ampcor.Ampcor import Ampcor
from isce.components.isceobj.Image.Image import Image
from isce.applications.stripmapApp import Insar
from lxml import etree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os
os.chdir("data")

In [ ]:
def stripmapApp(step):
    insar = Insar(name = "stripmapApp",cmdline=["stripmapApp.xml","--steps", step])
    insar.configure()
    status = insar.run()
    raise SystemExit(status)

stripmapApp("--end=preprocess")

得到`reference_slc/reference.slc.xml`后，将该文件中的行数全部换为18105，列数全部换为21642，得到现在的`reference.slc.xml`文件。同理，替换好`reference.vrt`，注意LineOffset等于列数*8。

若改变了目录结构或者文件名，还需要改动`<property name="extra_file_name" />`和`<property name="file_name" />`的值。生成高程数据`z.rdr.full.xml`时需要注意。

天绘二号对应的`reference.slc`可以通过以下函数得到

In [ ]:
def cos2slc(cos_path):
    endian = 'little'  # TerraSAR-X的.cos是以大端序保存的，但天会二号是小端序。
    with open(cos_path, 'rb') as cos:
        s = 4
        Bytes_In_Burst = int.from_bytes(
            cos.read(s), byteorder=endian)  # BIB, 一个burst的总字节数
        Range_Sample_Relative_Index = int.from_bytes(
            cos.read(s), byteorder=endian)
        # RSRI, 距离向采样点索引值，给出了在一个虚拟栅格上每一行距离向上第一个采样点相对于图像中心参考点零多普勒时刻位置的索引值
        # RS, 距离向采样点数，该值在所有burst中保持一致
        Range_Samples = int.from_bytes(cos.read(s), byteorder=endian)
        Azimuth_Samples = int.from_bytes(
            cos.read(s), byteorder=endian)  # AS, 方位向采样点数
        Burst_Index = int.from_bytes(
            cos.read(s), byteorder=endian)  # BI, burst个数的索引值
        Rangeline_Total_Number_of_Bytes = int.from_bytes(
            cos.read(s), byteorder=endian)  # RTNB, 一行距离向的总字节数，包含注释信息
        Total_Number_of_Lines = int.from_bytes(cos.read(s), byteorder=endian)
        # TNL, 方位向上的长度，即行数，包含注释信息。该值只在文件第一行给出，方便文件读取和其他burst替换。因此整个文件大小可以通过RTNB×TNL得出
        # 很遗憾，上述数据中的BIB和RTNB是错误的，TNL也说不上对但暂且不管，并非读取错误而是数据本身错误。需要进行修正
        annotation_lines, annotation_columns, offset = 4 + 1, 2, -8
        # 天绘cos数据内的元数据是开头5行而不是开头4行，第5行还少了8字节，导致元数据的形状不是一个矩形
        Rangeline_Total_Number_of_Bytes = s * \
            (Range_Samples + annotation_columns)  # 真正的RTNB
        # 原始的RTNB = 2 * 真RTNB - 8，等于说除了Annotation（每行2字节）外长度增大至2倍，可能是设计者对short型理解有误
        # BIB = 原始的RTNB * TNL，因此BIB跟着RTNB出错
        # 真正的体积为 (TNL * 真RTNB) + (真RTNB - 8)，并不是一个矩形。
        cos.seek(annotation_lines * Rangeline_Total_Number_of_Bytes + offset)
        dtype = np.dtype(np.int16).newbyteorder(endian)
        res = np.float32(np.frombuffer(cos.read(Rangeline_Total_Number_of_Bytes * Azimuth_Samples), dtype=dtype).astype(
            np.dtype(np.int16).newbyteorder('big')).reshape(Azimuth_Samples, (Range_Samples + annotation_columns) * 2)[
            :, 4:])
        return res[:, ::2]+res[:, 1::2]*1j

这样天绘二号数据就算处理完成了。

为了生成`z.rdr.full`高程数据，有两种技术路线可以走：

1. 修改`referemce_slc.xml`。该文件在`reference.slc.xml`的上一级目录中，包含了时间和轨道等数据。本项目的`data/reference_slc.xml`已经参照`tianhui.par`完成了替换。

2. 自己生成高程数据。需要自行计算影像上每一像元的地理坐标。可以参考以下代码

In [ ]:
from multiprocessing import Pool
from joblib import dump, load
import numpy as np

Const_a = 6378137.0000
Const_b = 6356752.3141
Const_c = 299792458
Const_e2 = 0.006694380035512704

def degree2rad(degree):
    return degree * np.pi / 180


def rad2degree(rad):
    return rad * 180 / np.pi

def wgs2ecs(lon, lat, H):
    lon = degree2rad(lon)
    lat = degree2rad(lat)
    N = Const_a / np.sqrt(1 - Const_e2 * np.sin(lat) ** 2)
    x = (N + H) * np.cos(lat) * np.cos(lon)
    y = (N + H) * np.cos(lat) * np.sin(lon)
    z = (N * (1 - Const_e2) + H) * np.sin(lat)
    return x, y, z

def ecs2wgs(x, y, z):
    p = np.sqrt(x ** 2 + y ** 2)
    theta = np.arctan(z * Const_a / (p * Const_b))
    lon = np.arctan2(y, x)
    lat = np.arctan(
        (z + Const_e2 / (1 - Const_e2) * Const_b * np.sin(theta) ** 3) / (p - Const_e2 * Const_a * np.cos(theta) ** 3))
    N = Const_a / np.sqrt(1 - Const_e2 * np.sin(lat) ** 2)
    H = p / np.cos(lat) - N
    lon, lat = rad2degree(lon), rad2degree(lat)
    return lon, lat, H

def t2states(first_time, interval, states, t):
    states = np.array(states)
    dt = t - first_time
    i = np.int16(dt // interval)
    rate = (dt - i) / interval
    rate = rate.reshape(rate.shape + (1,))
    return states[i] * (1 - rate) + states[i + 1] * rate

def dot(m1, m2):
    if m1.ndim < m2.ndim:
        m1, m2 = m2, m1
    while m2.ndim < m1.ndim:
        m2 = np.expand_dims(m2, axis=-2)
    shape = m1.shape[:-1]
    return (np.expand_dims(m1, axis=-2) @ np.expand_dims(m2, axis=-1)).reshape(shape)

t_parts, R_parts = 1, 1
par, dem = None, None

def asf(t_count):
    t0 = par['start_time']
    dt = par['azimuth_line_time']
    nt = int(par['azimuth_lines'])
    t = t0 + np.arange(0, nt) * dt
    R0 = par['near_range_slc']
    dR = par['range_pixel_spacing']
    nR = int(par['range_samples'])
    R = R0 + np.arange(0, nR) * dR
    freq = par['radar_frequency']

    lamb = Const_c / freq
    first_time = par['time_of_first_state_vector']
    interval = par['state_vector_interval']
    pos = par['state_vector_position']
    vel = par['state_vector_velocity']
    doppler = par['doppler_polynomial']
    R_center = par['center_range_slc']
    # alpha、beta迭代的初始参数，从影像中心像元的经纬度开始迭代
    L = par['center_longitude']
    B = par['center_latitude']
    H = dem.find(L, B)
    X_T, Y_T, Z_T = wgs2ecs(L, B, H)
    R_T2 = X_T ** 2 + Y_T ** 2 + Z_T ** 2

    for R_count in range(R_parts):
        print('开始处理 %d / %d' % (t_count * R_parts + R_count + 1, t_parts * R_parts))
        # 分到该块慢时间、斜距
        pt = t[nt * t_count // t_parts: nt * (t_count + 1) // t_parts]
        pR = R[nR * R_count // R_parts: nR * (R_count + 1) // R_parts]
        npt, npR = len(pt), len(pR)
        alpha_error, beta_error = np.empty((npt, npR), dtype=np.float16), np.empty((npt, npR), dtype=np.float16)
        f_d = doppler[0] + doppler[1] * (pR - R_center)
        # 计算卫星的位置、速度向量以及模方便调用
        R_S = t2states(first_time, interval, pos, pt)
        V_S = t2states(first_time, interval, vel, pt)
        R_S_norm = np.linalg.norm(R_S, axis=-1)
        R_S_norm2 = R_S_norm ** 2
        V_S_norm = np.linalg.norm(V_S, axis=-1)
        X_S, Y_S, Z_S = R_S[:, 0:1], R_S[:, 1:2], R_S[:, 2:3]
        X_S2, Y_S2, Z_S2 = X_S ** 2, Y_S ** 2, Z_S ** 2
        # 计算旋转矩阵
        z_hat = R_S / R_S_norm
        y_hat = np.cross(R_S, V_S)
        y_hat /= np.linalg.norm(y_hat, axis=-1)
        x_hat = np.cross(y_hat, z_hat)
        M = np.array([x_hat, y_hat, z_hat]).swapaxes(0, 1)
        M = np.repeat(np.expand_dims(M, 1), npR, 1)

        # 计算方向向量e

        def cal_e(alpha, beta):
            e = np.array([np.sin(alpha), -np.cos(alpha) * np.sin(beta), -np.cos(alpha) * np.cos(beta)])
            e = np.rollaxis(e, 0, 3)
            e = (np.expand_dims(e, 2) @ M).reshape(e.shape)
            return e

        # 初始化参数，alpha为0，计算beta
        alpha = np.zeros((npt, npR))
        beta = np.arccos((R_S_norm2 + pR ** 2 - R_T2) / (2 * R_S_norm * pR))

        # beta迭代函数
        def cal_beta(alpha, beta, beta_times, rate=1.5):
            for i in range(beta_times):
                # 步骤1，计算beta角对应斜距与真实斜距之差
                e = cal_e(alpha, beta)
                R_T = np.expand_dims(R_S, 1) + pR.reshape(1, npR, 1) * e
                lon, lat, _ = ecs2wgs(R_T[:, :, 0], R_T[:, :, 1], R_T[:, :, 2], rad=False)
                H = dem.find(lon, lat)
                e_X, e_Y, e_Z = e[:, :, 0], e[:, :, 1], e[:, :, 2]
                aH2, bH2 = (Const_a + H) ** 2, (Const_b + H) ** 2
                A1 = (e_X ** 2 + e_Y ** 2) * bH2 + e_Z ** 2 * aH2
                A2 = 2 * ((X_S * e_X + Y_S * e_Y) * bH2 + Z_S * e_Z * aH2)
                A3 = (X_S2 + Y_S2) * bH2 + Z_S2 * aH2 - aH2 * bH2
                R1 = (-A2 - np.sqrt(A2 ** 2 - 4 * A1 * A3)) / (2 * A1)
                if i == beta_times - 1:
                    delta_R = pR - R1
                    beta_error[:, :] = np.float16(delta_R)
                # 步骤2，计算入射角，从而修正beta
                sin_eta = R_S_norm * np.sin(beta) / np.linalg.norm(R_T, axis=2)
                tan_eta = sin_eta / np.sqrt(1 - sin_eta ** 2)
                d_beta = (1 - R1 / pR) / tan_eta
                beta += d_beta / rate
            return beta

        # alpha迭代函数
        def cal_alpha(alpha, beta, alpha_times):
            for i in range(alpha_times):
                e = cal_e(alpha, beta)
                d_alpha = (lamb * f_d / 2 - dot(e, np.expand_dims(V_S, 1))) / V_S_norm
                if i == alpha_times - 1:
                    alpha_error[:, :] = np.float16(d_alpha * pR)
                alpha += d_alpha
            return alpha

        for i in range(2):
            alpha = cal_alpha(alpha, beta, 2)
            beta = cal_beta(alpha, beta, 12)
        alpha = cal_alpha(alpha, beta, 2)
        e = cal_e(alpha, beta)
        R_T = np.expand_dims(R_S, 1) + pR.reshape(1, npR, 1) * e
        lon, lat, _ = ecs2wgs(R_T[:, :, 0], R_T[:, :, 1], R_T[:, :, 2], rad=False)
        path = '%d_%d.pkl' % (t_count, R_count)
        dump(lon, path.format('lon'))
        dump(lat, path.format('lat'))


def makeup():
    nt = int(par['azimuth_lines'])
    nR = int(par['range_samples'])
    ans = np.empty((nt, nR), dtype=np.float64)
    h = 0
    for i in range(t_parts):
        w = 0
        for j in range(R_parts):
            data = load('%d_%d.pkl' % (i, j))
            ans[h:h + data.shape[0], w:w + data.shape[1]] = data
            w += data.shape[1]
            if j == R_parts - 1:
                h += data.shape[0]
    return ans



if __name__ == '__main__':
    p = Pool(4)
    for t_count in range(t_parts):
        p.apply(asf, args=(t_count,))
    p.close()
    p.join()
    # makeup()

计算出每个像元的地理坐标后，利用坐标对DEM数据进行采样，得到`z.rdr.full`。其头文件`z.rdr.full.vrt`与`z.rdr.full.xml`都与`reference`的对应文件基本一致，只是`xml`的`<property name="data_type">`需要改为"DOUBLE"等值。

若是处理TerraSAR-X数据，就不需要修改`referemce_slc.xml`或是进行这么复杂的计算，调用stripmapApp()时参数取"--end=topo"即可。这样天绘二号数据就与TerraSAR-X数据到了同一起跑线上。接下来是利用`z.rdr.full`生成模拟强度影像。

In [ ]:
def simAmp(dem_path="z.rdr.full.xml",sim_path="sim.rdr"):
    # 注意不能取.amp后缀名，否则会被mdx当作是双通道数据
    hgtImage = createImage()
    hgtImage.load(dem_path)
    hgtImage.setAccessMode('read')
    hgtImage.createImage()

    simImage = createImage()
    simImage.setFilename(sim_path)
    simImage.dataType = 'FLOAT'
    simImage.setAccessMode('write')
    simImage.setWidth(hgtImage.getWidth())
    simImage.createImage()

    objShade = Simamplitude()
    stdWriter = create_writer("log","",False)
    objShade.setStdWriter(stdWriter)
    objShade.simamplitude(hgtImage, simImage, shade=3.0)  # shade=3.0是将模拟影像的强度增大到计算值的3倍

    simImage.renderHdr()
    hgtImage.finalizeImage()
    simImage.finalizeImage()

最后是图像配准

In [ ]:
def file_type(path):
    type = etree.parse(path + ".xml").xpath("/imageFile/property[@name='data_type']/value[1]")[0].text.lower()
    if type == "cfloat":
        return "complex"
    elif type == "float":
        return "real"
    else:
        raise ValueError("未知的类型")


def main(ref_path, sec_path,windowSize=64,searchSize=100,azoffset=0, rgoffset =0):
    ref = Image()
    ref.load(ref_path + '.xml')
    ref.setAccessMode('READ')
    ref.createImage()

    sec = Image()
    sec.load(sec_path + '.xml')
    sec.setAccessMode('READ')
    sec.createImage()


    objOffset = Ampcor(name='')
    objOffset.configure()
    objOffset.setAcrossGrossOffset(rgoffset)
    objOffset.setDownGrossOffset(azoffset)
    objOffset.setWindowSizeWidth(windowSize)
    objOffset.setWindowSizeHeight(windowSize)
    objOffset.setSearchWindowSizeWidth(searchSize)
    objOffset.setSearchWindowSizeHeight(searchSize)

    nearestDistance = 101
    margin = 2*objOffset.searchWindowSizeWidth + objOffset.windowSizeWidth
    offAc = max(nearestDistance,-rgoffset) + margin
    offDn = max(nearestDistance,-azoffset) + margin
    lastAc = int(min(ref.getWidth(), sec.getWidth() - offAc) - margin)
    lastDn = int(min(ref.getLength(), sec.getLength() - offDn) - margin)

    objOffset.setFirstSampleAcross(offAc)
    objOffset.setLastSampleAcross(lastAc)
    objOffset.setFirstSampleDown(offDn)
    objOffset.setLastSampleDown(lastDn)

    nAcross = 60
    nDown = 60
    objOffset.setNumberLocationAcross(nAcross)
    objOffset.setNumberLocationDown(nDown)
    objOffset.setFirstPRF(1.0)
    objOffset.setSecondPRF(1.0)
    objOffset.setImageDataType1(file_type(ref_path))
    objOffset.setImageDataType2(file_type(sec_path))

    objOffset.ampcor(ref, sec)

    ref.finalizeImage()
    sec.finalizeImage()

    field = objOffset.getOffsetField()
    return field

使用以下代码可以对field进行滤波

In [ ]:
def analyze(field, snr=5):
    if snr != None:
        for distance in [10,5,3,1]:
            objOff = createOffoutliers()
            objOff.wireInputPort(name='offsets', object=field)
            objOff.setSNRThreshold(snr)
            objOff.setDistance(distance)
            objOff.setStdWriter(create_writer("log","",True,filename="isce.log"))
            objOff.offoutliers()
            field = objOff.getRefinedOffsetField()
            # print('%d points left'%(len(field._offsets)))
    azrgOrder=0
    azazOrder=1
    rgrgOrder=1
    rgazOrder=0
    aa, _ = field.getFitPolynomials(azimuthOrder=azazOrder, rangeOrder=azrgOrder, usenumpy=True)
    _, rr = field.getFitPolynomials(azimuthOrder=rgazOrder, rangeOrder=rgrgOrder, usenumpy=True)
    aa, rr = np.array(aa._coeffs), np.array(rr._coeffs)
    res = np.array(field.unpackOffsets())
    # res是所有样本点，aa是方位向偏移，rr是距离向偏移。
    return res, aa, rr